### インストール・初期設定等

In [1]:
!pip install -q transformers ipadic fugashi
!python -V
!rm -rf sample_data/
from google.colab import drive

drive.mount("/content/drive")

Python 3.8.10
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 標準ライブラリ
import gc
import glob
import json
import os
import pickle
import random
import re
import unicodedata

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertModel

In [3]:
# 前処理
def preprocessing(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[w/:%#$&?()~.=+-…]+[:]? ", "", text)
    text = re.sub(r"(^RT )", "", text)
    text = text.lower()  # 小文字化
    text = re.sub("\r", "", text)  # \r\nをdelete
    text = re.sub("\n", "", text)  # \r\nをdelete
    text = re.sub(r"\d+", "", text)  # 数字列をdelete
    ZEN = "".join(chr(0xFF01 + i) for i in range(94))  # 全角記号一覧
    HAN = "".join(chr(0x21 + i) for i in range(94))  # 半角記号一覧
    ETC = "".join(chr(0x3000 + i) for i in range(30))  # その他主要そうな記号
    text = text.translate(str.maketrans(ZEN, HAN))  # 全角記号を半角記号に置換
    FIXED_HAN = re.sub(r"[\w]+", "", HAN)
    return re.sub("[" + "~" + "*" + "＊" + ETC + FIXED_HAN + "]", " ", text)  # 記号を消す


# JSONからdf作成
def JSONtoDF(path):
    with open(path) as f:
        j = json.load(f)
        return pd.json_normalize(data=j["data"]).drop(
            ["created_at", "edit_history_tweet_ids"], axis=1
        )


# 上記2関数を組み合わせてcorpusを用意
def prepareCorpus(path_list):
    res = pd.DataFrame()
    for path in path_list:
        df = JSONtoDF(path)
        df["preprocessedText"] = [preprocessing(text) for text in df["text"]]
        res = pd.concat([res, df])
    return res.reset_index(drop=True)


def prepare_target_data(target_data_path=""):
    if not target_data_path or not os.path.exists("/content/" + target_data_path):
        print("Exit from function because the file doesn't exist")
        return
    with open(target_data_path) as f:
        target_data = json.load(f)
    target_keys = ["old", "new"]
    for key in target_data["data"]:
        item = target_data["data"][key]
        for target_key in target_keys:
            if not len([i["sentence"] for i in item[target_key]]):
                continue
            input = tokenizer(
                [unicodedata.normalize("NFC", i["sentence"]) for i in item[target_key]],
                return_tensors="pt",
                max_length=128,
                truncation=True,
                padding="max_length",
            )
            input = {k: v.to(device) for k, v in input.items()}
            outputs = model(**input)
            for i, v in zip(item[target_key], outputs.last_hidden_state):
                i["vector"] = torch.mean(v, 0).detach().tolist()
    with open(target_data_path, "w", encoding="utf-8") as f:
        json.dump(target_data, f, ensure_ascii=False)


def get_vectors(
    text_list=[],
    target_word=None,
    output_path=None,
    sequence_length=256,
    buffer_size=512,
    layer_range=(0, 12),
):
    print("------------------------")

    # check params
    if not target_word or not output_path or not len(text_list):
        print("Exit from function due to improper parameters")
        return

    # check outputs
    if not os.path.exists(os.path.dirname(output_path)):
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
    if os.path.exists("/content/" + output_path):
        print("Exit from function because the file already exists")
        return

    print("Start process : get_vectors")

    # initialize
    vectors = []
    start_index = 0
    text_list = list(filter(lambda x: target_word in x, text_list))
    while start_index < len(text_list):
        input = tokenizer(
            text_list[start_index : start_index + buffer_size],
            return_tensors="pt",
            max_length=sequence_length,
            truncation=True,
            padding="max_length",
        )
        input = {k: v.to(device) for k, v in input.items()}
        with torch.no_grad():
            outputs = model(**input, output_hidden_states=True)
            outputs_vectors = torch.mean(
                torch.stack(outputs.hidden_states[layer_range[0] : layer_range[1]]), 0
            )
            ids = tokenizer.encode(target_word)
            for i, v in tqdm(zip(input["input_ids"], outputs_vectors)):
                if ids[1] not in list(i):
                    continue
                pos = list(i).index(ids[1])
                vectors.append(
                    {
                        "position": (pos, pos + len(ids) - 2),
                        "vector": torch.mean(v, 0).detach().tolist(),
                    }
                )
        start_index += buffer_size

    print("Finish process : {} / {}".format(target_word, len(vectors)))
    with open(output_path, "wb") as f:
        pickle.dump(vectors, f)


def compare_sentences(
    vectors_path="", target_data={}, target_word="", result_data_path="", year=""
):
    print("Start process : compare_sentences")
    with open(vectors_path, "rb") as f:
        vectors = pickle.load(f)
    with open(result_data_path) as f:
        result_data = json.load(f)
        result_data.setdefault("data",{})
        result_data["data"].setdefault(str(target_word),{})
        result_data["data"][target_word][str(year)] = {"total": 0, "old": 0, "new": 0}
    # 文章単位での判別
    item = result_data["data"][target_word][year]
    item["total"] = len(vectors)
    for v in vectors:
        similarity_to_old = torch.mean(
            torch.tensor(
                [
                    torch.cosine_similarity(
                        torch.tensor(v["vector"]), torch.tensor(old["vector"]), 0
                    )
                    for old in target_data["data"][target_word]["old"]
                ]
            ),
            0,
        )
        similarity_to_new = torch.mean(
            torch.tensor(
                [
                    torch.cosine_similarity(
                        torch.tensor(v["vector"]), torch.tensor(new["vector"]), 0
                    )
                    for new in target_data["data"][target_word]["new"]
                ]
            ),
            0,
        )
        if similarity_to_new <= similarity_to_old:
            item["old"] += 1
        else:
            item["new"] += 1
    print(
        "Finish process : {:.2%} / {:.2%}".format(
            item["old"] / item["total"], item["new"] / item["total"]
        )
    )
    with open(result_data_path, "w", encoding="utf-8") as f:
        json.dump(result_data, f, ensure_ascii=False)

### 主要処理

In [4]:
# @title  { vertical-output: true, form-width: "35%", display-mode: "both" }

# 初期設定群
base_dir = "drive/MyDrive/script/bert/"  # @param {type:"string"}
with open(base_dir + "target_data.json", "r", encoding="utf-8") as f:
    target_data = json.load(f)

target_words = [unicodedata.normalize("NFC", w) for w in target_data["meta"]["target_words"]]

path_list = glob.glob(base_dir + "twitter-corpus/*.json")
path_list.sort()
oparation = [
    {
        "word": w,
        "year": y,
        "path_list": list(
            filter(
                lambda x: w in unicodedata.normalize("NFC", x) and str(y) in x,
                path_list,
            )
        ),
    }
    for w in target_words
    for y in range(2007, 2021)
]

target = "cl-tohoku/bert-base-japanese-whole-word-masking"  # @param {type:"string"}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertJapaneseTokenizer.from_pretrained(target, tokenize_chinese_chars=False)
model = BertModel.from_pretrained(target).to(device)


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# target_dataの例文が変わる度にこのセルを実行する
prepare_target_data(base_dir + "target_data.json")

In [5]:
# 分散表現を取得する
for o in oparation:
    if not len(o["path_list"]):
        print("Pass this loop due to empty of path_list")
        continue
    sequence_length = 256
    layer_range = (0, 12)
    target_word = unicodedata.normalize("NFC", o["word"])
    output_path = (
        base_dir
        + "word-vectors/"
        + str(sequence_length)
        + "-"
        + str(layer_range[0])
        + "-"
        + str(layer_range[1])
        + "/"
        + str(target_word)
        + "-"
        + str(o["year"])
        + ".pickle"
    )
    get_vectors(
        text_list=prepareCorpus(o["path_list"])["preprocessedText"].values.tolist(),
        target_word=target_word,
        output_path=output_path,
        sequence_length=sequence_length,
        buffer_size=512,
        layer_range=layer_range,
    )
    compare_sentences(
        vectors_path=output_path,
        target_data=target_data,
        target_word=unicodedata.normalize("NFC", o["word"]),
        result_data_path=base_dir+"result_data.json",
        year=str(o["year"])
    )

------------------------
Start process : get_vectors


512it [00:02, 238.76it/s]
512it [00:02, 240.51it/s]
396it [00:01, 244.08it/s]


Finish process : 失笑 / 1420
Start process : compare_sentences
Finish process : 61.27% / 38.73%
------------------------
Start process : get_vectors


512it [00:02, 229.69it/s]
512it [00:02, 235.10it/s]
512it [00:02, 235.48it/s]
512it [00:02, 236.89it/s]
512it [00:02, 236.19it/s]
27it [00:00, 251.94it/s]


Finish process : 失笑 / 2587
Start process : compare_sentences
Finish process : 72.94% / 27.06%
------------------------
Start process : get_vectors


...

Finish process : 草 / 9844
Start process : compare_sentences
Finish process : 64.53% / 35.47%
------------------------
Start process : get_vectors


512it [00:02, 240.63it/s]
512it [00:02, 247.57it/s]
512it [00:02, 244.59it/s]
512it [00:02, 247.11it/s]
512it [00:02, 242.55it/s]
512it [00:02, 243.48it/s]
512it [00:02, 245.83it/s]
512it [00:02, 244.55it/s]
512it [00:02, 243.55it/s]
512it [00:02, 245.17it/s]
512it [00:02, 250.79it/s]
512it [00:02, 248.02it/s]
512it [00:02, 246.30it/s]
512it [00:02, 249.74it/s]
512it [00:02, 242.85it/s]
512it [00:02, 247.68it/s]
512it [00:02, 250.59it/s]
512it [00:02, 251.67it/s]
326it [00:01, 249.19it/s]


Finish process : 草 / 9540
Start process : compare_sentences
Finish process : 22.94% / 77.06%
------------------------
Start process : get_vectors


512it [00:02, 247.69it/s]
512it [00:02, 251.79it/s]
512it [00:02, 250.10it/s]
512it [00:02, 248.31it/s]
512it [00:02, 246.86it/s]
512it [00:02, 252.25it/s]
512it [00:02, 248.33it/s]
512it [00:02, 249.66it/s]
512it [00:02, 250.26it/s]
512it [00:02, 249.34it/s]
512it [00:02, 251.73it/s]
512it [00:02, 253.21it/s]
512it [00:02, 250.34it/s]
512it [00:02, 253.51it/s]
512it [00:02, 246.14it/s]
512it [00:02, 251.35it/s]
512it [00:02, 251.53it/s]
512it [00:02, 252.78it/s]
191it [00:00, 252.53it/s]


Finish process : 草 / 9403
Start process : compare_sentences
Finish process : 22.41% / 77.59%


In [8]:
# 結果確認
with open(base_dir+"result_data.json") as f:
    result_data = json.load(f)
for word in result_data["data"]:
    print("-------------- "+word+" --------------")
    item = result_data["data"][word]
    for year in item:
        i = item[year]
        print("old: {:.2%} / new: {:.2%}".format(i["old"] / i["total"], i["new"] / i["total"]))

-------------- 失笑 --------------
old: 61.27% / new: 38.73%
old: 72.94% / new: 27.06%
old: 78.09% / new: 21.91%
old: 75.92% / new: 24.08%
old: 73.05% / new: 26.95%
old: 74.53% / new: 25.47%
old: 63.14% / new: 36.86%
old: 75.23% / new: 24.77%
old: 71.60% / new: 28.40%
old: 77.51% / new: 22.49%
old: 72.18% / new: 27.82%
old: 78.14% / new: 21.86%
old: 75.99% / new: 24.01%
old: 85.07% / new: 14.93%
-------------- なし崩し --------------
old: 85.71% / new: 14.29%
old: 84.52% / new: 15.48%
old: 78.07% / new: 21.93%
old: 86.72% / new: 13.28%
old: 84.63% / new: 15.37%
old: 85.79% / new: 14.21%
old: 95.98% / new: 4.02%
old: 86.16% / new: 13.84%
old: 84.28% / new: 15.72%
old: 86.50% / new: 13.50%
old: 83.18% / new: 16.82%
old: 89.20% / new: 10.80%
old: 96.51% / new: 3.49%
old: 58.44% / new: 41.56%
-------------- なしくずし --------------
old: 100.00% / new: 0.00%
old: 93.02% / new: 6.98%
old: 96.38% / new: 3.62%
old: 97.41% / new: 2.59%
old: 97.05% / new: 2.95%
old: 97.22% / new: 2.78%
old: 97.09% / new: 

### 保存後処理

In [ ]:
# formatter
!pip install -q black[jupyter]
!black "/content/drive/MyDrive/Colab Notebooks/bertClustering.ipynb"

### その他

In [ ]:
from tensorflow.python.client import device_lib
import tensorflow as tf
from psutil import virtual_memory

# RAMのサイズをcheck
ram_gb = virtual_memory().total / 1e9
print("Your runtime has {:.1f} gigabytes of available RAM\n".format(ram_gb))

if ram_gb < 20:
    print("Not using a high-RAM runtime")
else:
    print("You are using a high-RAM runtime!")
# GPUの数をcheck
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
# Check GPU recognized
print(device_lib.list_local_devices())